In [ ]:
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

In [ ]:
training_data = pd.read_csv('../../numerai_datasets/numerai_training_data.csv', header=0)

In [ ]:
prediction_data = pd.read_csv('../../numerai_datasets/numerai_tournament_data.csv',  header=0)

In [ ]:
#feature1 = tf.feature_column.numeric_column('feature1')
#feature2 = tf.feature_column.numeric_column('feature2')
#feature3 = tf.feature_column.numeric_column('feature3')
#feature4 = tf.feature_column.numeric_column('feature4')
#feature5 = tf.feature_column.numeric_column('feature5')
#feature6 = tf.feature_column.numeric_column('feature6')
#feature7 = tf.feature_column.numeric_column('feature7')
feature8 = tf.feature_column.numeric_column('feature8')
feature9 = tf.feature_column.numeric_column('feature9')
#feature10 = tf.feature_column.numeric_column('feature10')
feature11 = tf.feature_column.numeric_column('feature11')
#feature12 = tf.feature_column.numeric_column('feature12')
feature13 = tf.feature_column.numeric_column('feature13')
#feature14 = tf.feature_column.numeric_column('feature14')
#feature15 = tf.feature_column.numeric_column('feature15')
feature16 = tf.feature_column.numeric_column('feature16')
#feature17 = tf.feature_column.numeric_column('feature17')
#feature18 = tf.feature_column.numeric_column('feature18')
#feature19 = tf.feature_column.numeric_column('feature19')
#feature20 = tf.feature_column.numeric_column('feature20')
#feature21 = tf.feature_column.numeric_column('feature21')
#feature22 = tf.feature_column.numeric_column('feature22')
#feature23 = tf.feature_column.numeric_column('feature23')
feature24 = tf.feature_column.numeric_column('feature24')
#feature25 = tf.feature_column.numeric_column('feature25')
#feature26 = tf.feature_column.numeric_column('feature26')
feature27 = tf.feature_column.numeric_column('feature27')
feature28 = tf.feature_column.numeric_column('feature28')
#feature29 = tf.feature_column.numeric_column('feature29')
#feature30 = tf.feature_column.numeric_column('feature30')
feature31 = tf.feature_column.numeric_column('feature31')
#feature32 = tf.feature_column.numeric_column('feature32')
#feature33 = tf.feature_column.numeric_column('feature33')
#feature34 = tf.feature_column.numeric_column('feature34')
#feature35 = tf.feature_column.numeric_column('feature35')
feature36 = tf.feature_column.numeric_column('feature36')
feature37 = tf.feature_column.numeric_column('feature37')
#feature38 = tf.feature_column.numeric_column('feature38')
#feature39 = tf.feature_column.numeric_column('feature39')
#feature40 = tf.feature_column.numeric_column('feature40')
#feature41 = tf.feature_column.numeric_column('feature41')
#feature42 = tf.feature_column.numeric_column('feature42')
#feature43 = tf.feature_column.numeric_column('feature43')
#feature44 = tf.feature_column.numeric_column('feature44')
#feature45 = tf.feature_column.numeric_column('feature45')
feature46 = tf.feature_column.numeric_column('feature46')
feature47 = tf.feature_column.numeric_column('feature47')
#feature48 = tf.feature_column.numeric_column('feature48')
#feature49 = tf.feature_column.numeric_column('feature49')
#feature50 = tf.feature_column.numeric_column('feature50')

In [ ]:
feat_cols = [
    feature8, feature9, feature11, feature13, feature16, feature24, feature27, 
    feature28, feature31, feature36, feature37, feature46, feature47]

In [ ]:
x_data = training_data[[
    'feature8', 'feature9', 'feature11', 'feature13', 'feature16', 'feature24', 'feature27', 'feature28', 
    'feature31', 'feature36', 'feature37', 'feature46', 'feature47']]

In [ ]:
labels = training_data['target']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.5)

In [ ]:
# Parameters
num_steps = 500 # Total steps to train
batch_size = 1024 # The number of samples per batch
num_classes = 2 # The 10 digits
num_features = 13 # 50 columns
num_trees = 10
max_nodes = 1000
display_step = 1

In [ ]:
# tf Graph input
x = tf.placeholder("float32", shape=[None, num_features])
y = tf.placeholder("float32", shape=[None])

In [ ]:
# Random Forest Parameters
hparams = tensor_forest.ForestHParams(num_classes=num_classes,
                                      num_features=num_features,
                                      num_trees=num_trees,
                                      max_nodes=max_nodes).fill()

In [ ]:
# Build the Random Forest
forest_graph = tensor_forest.RandomForestGraphs(hparams)

In [ ]:
# Get training graph and loss
train_op = forest_graph.training_graph(x, y)
loss_op = forest_graph.training_loss(x, y)

In [ ]:
# Initialize the variables (i.e. assign their default value) and forest resources
init_vars = tf.group(tf.global_variables_initializer(),
    resources.initialize_resources(resources.shared_resources()))

In [ ]:
# Measure the accuracy
infer_op, _, _ = forest_graph.inference_graph(x)
correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Start TensorFlow session
#sess = tf.Session()

In [ ]:
# Run the initializer
#sess.run(init_vars)

In [ ]:
X_train.shape

In [ ]:
import numpy as np

In [ ]:
y_train = np.array([y_train, -(y_train-1)]).T

In [ ]:
y_test = np.array([y_test, -(y_test-1)]).T

In [ ]:
y_train.shape

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)

In [ ]:
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init_vars)
    # Training cycle
    for epoch in range(num_steps):
        avg_cost = 0.
        total_batch = int(len(X_train)/batch_size)
        X_batches = np.array_split(X_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={x: batch_x,
                                                          y: batch_y})
            if i % 50 == 0 or i == 1:
                acc = sess.run(accuracy_op, feed_dict={x: batch_x, y: batch_y})
                print('Step %i, Loss: %f, Acc: %f' % (i, c, acc))
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            

        
    print("Optimization Finished!")

In [ ]:
# Training
for i in range(1, num_steps + 1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0.
    total_batch = int(len(X_train)/batch_size)
    X_batches = np.array_split(X_train, total_batch)
    Y_batches = np.array_split(y_train, total_batch)
    # Loop over all batches
    for j in range(total_batch):
        batch_x, batch_y = X_batches[j], Y_batches[j]
        print(batch_x)
        _, l = sess.run([train_op, loss_op], feed_dict={x: batch_x, y: batch_y})
        
        # Compute average loss
        avg_cost += l / total_batch
    # Display logs per epoch step
    if i % display_step == 0:
        print('1. Step %i, Loss: %f' % (i, l))
   
    #if i % 50 == 0 or i == 1:
    #    acc = sess.run(accuracy_op, feed_dict={x: batch_x, y: batch_y})
    #    print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

In [ ]:
list(result)